In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from brian2 import *
from scipy.stats import circstd,circmean

In [1]:
def mean_over_phi_start(filename_res,carrier):
    res=load(filename_res)
    nbr_param_sets=res.size/res.shape[0]
    res_flat=res.reshape((res.shape[0],nbr_param_sets,res.shape[6]))
    res_flat_average=np.zeros((res.shape[0],nbr_param_sets))
    sd_flat=np.zeros((res.shape[0],nbr_param_sets))
    for i in xrange(nbr_param_sets):
        if not np.all(res_flat[:,i,:]):
            res_flat_average[:,i]=nan
        else:
            res_flat_average[:,i]=circmean(res_flat[:,i,:],high=360,axis=1)
            sd_flat[:,i]=sqrt(circvar(res_flat[:,i,:],high=360,axis=1))
    res_average=res_flat_average.reshape((res.shape[0],res.shape[1],res.shape[2],res.shape[3],res.shape[4],res.shape[5]))
    sd=sd_flat.reshape((res.shape[0],res.shape[1],res.shape[2],res.shape[3],res.shape[4],res.shape[5]))
    save('phase_'+str(carrier)+'.npy',res_average)
    save('sd_'+str(carrier)+'.npy',sd_average)

In [ ]:
def max_error(filename_phase,carrier):
    if carrier==200:
        phase_data=np.array([180,180,180,180,180])
    else:
        phase_data=np.array([37, 40, 62, 83, 115])
    phase=load(filename_phase)
    nbr_param_sets=phase.size/phase.shape[0]
    phase_flat=phase.reshape((phase.shape[0],nbr_param_sets))
    
    max_error_flat=np.zeros(nbr_param_sets)
   
    for i in xrange(nbr_param_sets) :
        if sum(isnan(phase_flat[:,i]))!=0:
            max_error_flat[i]=nan
        else:
            max_error_flat[i]=amax(abs(phase_flat[:,i]-phase_data))
    max_error=max_error_flat.reshape((1,res.shape[1],res.shape[2],res.shape[3],res.shape[4],res.shape[5]))

    save('max_error_'+str(carrier)+'.npy',max_error)

In [ ]:
def max_error_combined(ind_param,filename_max_err_500,filename_max_err_200,err_weight=1.0/3):
    param_order=['tau_b','sigma_k','gamma','k','m','bipd']
    num_freq=2
    max_err_200=load(filename_max_err_200)
    max_err_500=load(filename_max_err_500)
    num_param=5
    
    if ind_param==5: # Case BIPD
        max_err_carrier_comb_all=np.zeros((max_err_500.shape[1],max_err_500.shape[2],
                                           max_err_500.shape[3],max_err_500.shape[4],max_err_500.shape[5]))   
        cp=0
        for i in xrange(max_err_500.shape[1]):
            for j in xrange(max_err_500.shape[2]):
                for k in xrange(max_err_500.shape[3]):
                    for l in xrange(max_err_500.shape[4]):
                        for m in xrange(max_err_500.shape[5]):
                            if isnan(max_err_200[0,i,j,k,l,m]) or isnan(max_err_500[0,i,j,k,l,m]):
                                max_err_carrier_comb_all[i,j,k,l,m]=nan
                                cp+=1
                            else:
                                max_err_carrier_comb_all[i,j,k,l,m]=amax([max_err_500[0,i,j,k,l,m],
                                                                          max_err_200[0,i,j,k,l,m]*1.0*err_weight])
    else: #other cases

        def indices_combined(ind_param,size):
            inds=np.zeros(size)
            for k in xrange(size):
                if k==ind_param:
                    inds[k]=k+1
                    inds[k+1]=k+1
                elif k>ind_param+1:
                    inds[k]=k
                elif k<ind_param:
                    inds[k]=k+1
            return tuple(inds)

        def indices_single(ind_param,size,ite):
            inds_200=[0]*(size)
            inds_500=[0]*(size)
            for p in xrange(size):
                if p==ind_param:
                    inds_200[p]=ite[p+1]
                    inds_500[p]=ite[p]
                elif p>ind_param:
                    inds_200[p]=ite[p+1]
                    inds_500[p]=ite[p+1]
                elif p<ind_param:
                    inds_200[p]=ite[p]
                    inds_500[p]=ite[p]
            inds_200=[0]+inds_200
            inds_500=[0]+inds_500
            return tuple(inds_200),tuple(inds_500)

        inds=indices_combined(ind_param,num_param+1)
        max_err_carrier_comb_all=np.zeros((max_err_500.shape[int(inds[0])],max_err_500.shape[int(inds[1])],max_err_500.shape[int(inds[2])],
                                           max_err_500.shape[int(inds[3])],max_err_500.shape[int(inds[4])],max_err_500.shape[int(inds[5])]))
        cp=0

        for i in xrange(max_err_500.shape[int(inds[0])]):
            for j in xrange(max_err_500.shape[int(inds[1])]):
                for k in xrange(max_err_500.shape[int(inds[2])]):
                    for l in xrange(max_err_500.shape[int(inds[3])]):
                        for m in xrange(max_err_500.shape[int(inds[4])]):
                            for n in xrange(max_err_200.shape[int(inds[5])]):
                                inds_200,inds_500=indices_single(ind_param,num_param,[i,j,k,l,m,n])
                                if isnan(max_err_200[inds_200]) or isnan(max_err_500[inds_500]):

                                    max_err_carrier_comb_all[i,j,k,l,m,n]=nan
                                    cp+=1
                                else:
                                    max_err_carrier_comb_all[i,j,k,l,m,n]=amax([max_err_500[inds_500],max_err_200[inds_200]*1.0*err_weight])


    save('max_error_combined_'+str(param_order[ind_param])+'.npy',max_err_carrier_comb_all) 
    save('nan_counter_'+str(param_order[ind_param])+'.npy',cp)
